In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import random
import math
import openpyxl

# 设置中文字体
rcParams['font.sans-serif'] = ['SimHei']
rcParams['axes.unicode_minus'] = False

# 定义不同情况的参数
cases = [
    {'零配件1次品率': 0.1, '零配件2次品率': 0.1, '成品次品率': 0.1, '零配件1检测成本': 2, '零配件2检测成本': 3,
     '成品检测成本': 3, '装配成本': 6, '市场售价': 56, '调换损失': 6, '拆解费用': 5},
    {'零配件1次品率': 0.2, '零配件2次品率': 0.2, '成品次品率': 0.2, '零配件1检测成本': 2, '零配件2检测成本': 3,
     '成品检测成本': 3, '装配成本': 6, '市场售价': 56, '调换损失': 6, '拆解费用': 5},
    {'零配件1次品率': 0.1, '零配件2次品率': 0.1, '成品次品率': 0.1, '零配件1检测成本': 2, '零配件2检测成本': 3,
     '成品检测成本': 3, '装配成本': 6, '市场售价': 56, '调换损失': 30, '拆解费用': 5},
    {'零配件1次品率': 0.2, '零配件2次品率': 0.2, '成品次品率': 0.2, '零配件1检测成本': 1, '零配件2检测成本': 1,
     '成品检测成本': 2, '装配成本': 6, '市场售价': 56, '调换损失': 30, '拆解费用': 5},
    {'零配件1次品率': 0.1, '零配件2次品率': 0.2, '成品次品率': 0.1, '零配件1检测成本': 8, '零配件2检测成本': 1,
     '成品检测成本': 2, '装配成本': 6, '市场售价': 56, '调换损失': 10, '拆解费用': 5},
    {'零配件1次品率': 0.05, '零配件2次品率': 0.05, '成品次品率': 0.05, '零配件1检测成本': 2, '零配件2检测成本': 3,
     '成品检测成本': 3, '装配成本': 6, '市场售价': 56, '调换损失': 10, '拆解费用': 40}
]

# 计算总成本和成品次品率的函数
def calc_total_cost(case, detect_parts1=True, detect_parts2=True, detect_final=True, dismantle=True):
    n_parts1 = 100  # 零配件1的数量
    n_parts2 = 100  # 零配件2的数量
    cost_parts1 = n_parts1 * case['零配件1检测成本'] if detect_parts1 else 0  # 零配件1的检测成本
    cost_parts2 = n_parts2 * case['零配件2检测成本'] if detect_parts2 else 0  # 零配件2的检测成本

    loss_parts1 = (n_parts1 * case['零配件1次品率']) * (case['装配成本']) if not detect_parts1 else 0  # 零配件1的次品损失
    loss_parts2 = (n_parts2 * case['零配件2次品率']) * (case['装配成本']) if not detect_parts2 else 0  # 零配件2的次品损失

    n_final_products = 100  # 成品数量
    cost_final = n_final_products * case['成品检测成本'] if detect_final else 0  # 成品的检测成本
    loss_final = (n_final_products * case['成品次品率']) * case['调换损失'] if not detect_final else 0  # 成品的次品损失

    dismantle_cost = n_final_products * case['拆解费用'] if dismantle else 0  # 拆解费用
    dismantle_revenue = (n_final_products * case['成品次品率']) * case['市场售价'] if dismantle else 0  # 拆解收益

    total_cost = (cost_parts1 + cost_parts2 + loss_parts1 + loss_parts2 +
                  cost_final + loss_final + dismantle_cost - dismantle_revenue)  # 总成本

    defective_rate = case['成品次品率'] * (1 if detect_final else 0.5)  # 成品次品率

    return total_cost, defective_rate

# 适应度函数：计算策略的总成本的负值（因为我们希望最小化总成本）
def fitness_function(strategy, case):
    total_cost, _ = calc_total_cost(case, **strategy)
    return -total_cost

# 初始化种群：生成初始策略组合
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        strategy = {
            "detect_parts1": random.choice([True, False]),
            "detect_parts2": random.choice([True, False]),
            "detect_final": random.choice([True, False]),
            "dismantle": random.choice([True, False])
        }
        population.append(strategy)
    return population

# 选择操作：选择适应度较高的个体进行繁殖
def selection(population, fitnesses, num_parents):
    parents = []
    for _ in range(num_parents):
        max_fitness_index = fitnesses.index(max(fitnesses))
        parents.append(population[max_fitness_index])
        fitnesses[max_fitness_index] = float('-inf')
    return parents

# 交叉操作：生成新的个体
def crossover(parent1, parent2):
    child = {}
    for key in parent1:
        child[key] = random.choice([parent1[key], parent2[key]])
    return child

# 变异操作：引入新的基因
def mutation(strategy, mutation_rate):
    for key in strategy:
        if random.random() < mutation_rate:
            strategy[key] = not strategy[key]
    return strategy

# 遗传算法主循环
def genetic_algorithm(case, population_size=100, num_generations=100, mutation_rate=0.01):
    population = initialize_population(population_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(strategy, case) for strategy in population]
        parents = selection(population, fitnesses, population_size // 2)
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutation(child, mutation_rate)
            new_population.append(child)
        population = new_population
    best_strategy = population[fitnesses.index(max(fitnesses))]
    return best_strategy

# 模拟退火算法
def simulated_annealing(case, initial_temp=10000, cooling_rate=0.95, stopping_temp=0.1):
    current_state = [True, True, True, True]  # [检测零件1, 检测零件2, 检测成品, 拆解]
    current_cost = calc_total_cost(case, *current_state)
    best_state = current_state
    best_cost = current_cost
    temp = initial_temp
    cost_history = []

    while temp > stopping_temp:
        new_state = current_state.copy()
        # 随机改变一个检测/拆解的布尔值
        index = random.randint(0, 3)
        new_state[index] = not new_state[index]

        new_cost = calc_total_cost(case, *new_state)

        # 如果新的成本更低，接受新状态
        if new_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - new_cost) / temp):
            current_state = new_state
            current_cost = new_cost

        # 更新最佳状态
        if new_cost < best_cost:
            best_state = new_state
            best_cost = new_cost

        # 降低温度
        temp *= cooling_rate
        cost_history.append(best_cost)

    return best_state, best_cost, cost_history

# 交叉验证
def cross_validation(cases, folds=5):
    results = []
    for i, case in enumerate(cases):
        print(f'Running case {i+1}/{len(cases)}...')
        
        # 遗传算法
        best_strategy_ga = genetic_algorithm(case)
        total_cost_ga, defective_rate_ga = calc_total_cost(case, **best_strategy_ga)
        print(f'遗传算法 - 最佳策略: {best_strategy_ga}, 总成本: {total_cost_ga}, 成品次品率: {defective_rate_ga}')

        # 模拟退火算法
        best_state_sa, best_cost_sa, cost_history_sa = simulated_annealing(case)
        total_cost_sa, defective_rate_sa = calc_total_cost(case, *best_state_sa)
        print(f'模拟退火算法 - 最佳策略: {best_state_sa}, 总成本: {total_cost_sa}, 成品次品率: {defective_rate_sa}')

        results.append({
            'case': i+1,
            'best_strategy_ga': best_strategy_ga,
            'total_cost_ga': total_cost_ga,
            'defective_rate_ga': defective_rate_ga,
            'best_state_sa': best_state_sa,
            'total_cost_sa': total_cost_sa,
            'defective_rate_sa': defective_rate_sa
        })

        # 绘制成本历史
        plt.plot(cost_history_sa, label=f'Case {i+1}')
    
    plt.title('Cost History Across Different Cases')
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.legend()
    plt.show()

    return results

# 执行交叉验证
results = cross_validation(cases)

# 输出结果到xlsx文件
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "B2策略"
ws.append(["策略编号", "遗传算法 - 策略描述", "遗传算法 - 总成本", "遗传算法 - 成品次品率", "模拟退火算法 - 策略描述", "模拟退火算法 - 总成本", "模拟退火算法 - 成品次品率"])
for i, result in enumerate(results):
    strategy_description_ga = (
        f"检测零配件1: {'是' if result['best_strategy_ga']['detect_parts1'] else '否'}, "
        f"检测零配件2: {'是' if result['best_strategy_ga']['detect_parts2'] else '否'}, "
        f"检测成品: {'是' if result['best_strategy_ga']['detect_final'] else '否'}, "
        f"拆解不合格成品: {'是' if result['best_strategy_ga']['dismantle'] else '否'}"
    )
    strategy_description_sa = (
        f"检测零配件1: {'是' if result['best_state_sa'][0] else '否'}, "
        f"检测零配件2: {'是' if result['best_state_sa'][1] else '否'}, "
        f"检测成品: {'是' if result['best_state_sa'][2] else '否'}, "
        f"拆解不合格成品: {'是' if result['best_state_sa'][3] else '否'}"
    )
    ws.append([i + 1, strategy_description_ga, result['total_cost_ga'], result['defective_rate_ga'], strategy_description_sa, result['total_cost_sa'], result['defective_rate_sa']])
wb.save('B2策略.xlsx')

Running case 1/6...
遗传算法 - 最佳策略: {'detect_parts1': False, 'detect_parts2': False, 'detect_final': False, 'dismantle': True}, 总成本: 120.0, 成品次品率: 0.05


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'